In [ ]:
:load LatexTable.hs
:load Logic.hs
import qualified LatexTable as T
import qualified Logic as L

# Given

In [ ]:
data MachineKind = MealyFSM | MooreFSM
fromZero = ((subtract 1 <$>) <$>)

transitions = fromZero [[2, 1, 5, 6, 2, 3], [4, 3, 1, 4, 6, 5], [6, 5, 3, 2, 4, 1]]
outputs = fromZero [[1, 2, 3, 3, 2, 1], [2, 3, 1, 2, 1, 3], [3, 1, 2, 1, 3, 2]]

machine = MealyFSM
checkInputs = [ [0, 0], [0, 1], [0, 1], [0, 1], [0, 1], [1, 0]
              , [1, 0], [1, 0], [0, 0], [0, 1], [0, 1], [0, 0]
              , [1, 0], [0, 0], [0, 0], [0, 0], [1, 0], [1, 0] ]
checkOutputs = [ [0, 0], [1, 0], [0, 0], [0, 1], [0, 1], [0, 0] 
               , [0, 0], [1, 0], [1, 0], [1, 0], [0, 0], [0, 0]
               , [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [0, 1] ] 

# Encoded states & transitions

In [ ]:
binaryWidth :: [Int] -> Int
binaryWidth = (+ 1) . floor . logBase 2.0 . fromIntegral . maximum

encodeBinary :: Int -> Int -> [Int]
encodeBinary width = padBinary width . reverse . revBinary
    where
        revBinary 0 = []
        revBinary n = (n `mod` 2) : revBinary (n `div` 2)
        padBinary width bs = replicate (width - length bs) 0 ++ bs

encodeBinaryAll :: [Int] -> [[Int]]
encodeBinaryAll ns = encodeBinary (binaryWidth ns) <$> ns

xrange = [0..(length transitions - 1)]
qrange = [0..maximum (head transitions)]
yrange = [0..(length outputs - 1)]
xwidth = binaryWidth [length transitions - 1]
qwidth = binaryWidth $ head transitions
ywidth = binaryWidth $ head outputs

transitionsEncoded = (encodeBinary qwidth <$>) <$> transitions
outputsEncoded = (encodeBinary ywidth <$>) <$> outputs

In [ ]:
statesEncodedHeader = statesHeaderCell : ((show =<<) <$> encodeBinaryAll qrange)
    where
        statesHeaderCell = "$" ++ mconcat (xs ++ "/" : qs) ++ "$"
        xs = ("x_" ++) . show <$> [1..xwidth]
        qs = ("Q_" ++) . show <$> [1..qwidth]

tabulateSignals' = T.render T.NoHeader T.DefaultCells statesEncodedHeader .
    T.prependRowHeader ((show =<<) . encodeBinary xwidth <$> xrange)
tabulateSignals ss footer = tabulateSignals' $ showEncodedRows ss ++ footer
    where showEncodedRows = (((show =<<) <$>) <$>)

-- Transitions encoded
putStrLn $ tabulateSignals transitionsEncoded []
    
-- Outputs encoded
makeFooter :: Char -> Int -> [String]
makeFooter var width = replicate (length qrange) yfooter
    where yfooter = "$" ++ mconcat ((var :) . ('_' :) . show <$> [1..width]) ++ "$"
putStrLn $ tabulateSignals outputsEncoded [makeFooter 'y' ywidth]

# Output DNF

In [ ]:
import Control.Monad (forM_)

type Enc = [Int] -- encoded signal

zipWithXQ :: [[Enc]] -> [(Enc, [(Enc, Enc)])] -- [(X, [(Q, signal)])]
zipWithXQ signals = zip (encodeBinaryAll xrange) $ zip (encodeBinaryAll qrange) <$> signals

outputsWithXQ :: [(Enc, [(Enc, Enc)])] -- [(X, [(Q, W)])]
outputsWithXQ = zipWithXQ outputsEncoded

dnf :: [(Enc, [(Enc, Enc)])] -> Int -> L.BoolTerm
dnf inputs windex = L.simplify $ L.Or $ inputTerms =<< inputs
    where
        inputTerms (x, ws) = L.And . go x <$> ws
        go x (q, w)
            | w !! windex == 1 = (term L.X <$> zip [1..] x) ++ (term L.Q <$> zip [1..] q)
            | otherwise        = []
        term ctr (i, signal) = if signal == 1 then ctr i else L.Not (ctr i)

renderDnf var vari dnf = var ++ "_" ++ show (vari + 1) ++ " = " ++
        L.render dnf ++ " = " ++ L.renderNumericDnf dnf

ydnfrange = [0..ywidth - 1]
outputDnfs = dnf outputsWithXQ <$> ydnfrange
forM_ (zip [0..] outputDnfs) $ \(y, ydnf) -> putStrLn $ renderDnf "y" y ydnf

# Evaluation

In [ ]:
-- qdnfs -> ydnfs -> state -> inputs -> outputs
-- length inputs == length outputs
-- length qdnfs == length state
-- length ydnfs == length $ head outputs
eval :: [L.BoolTerm] -> [L.BoolTerm] -> Enc -> [Enc] -> [Enc]
eval qdnfs ydnfs initState inputs = reverse $ go initState inputs []
    where
        go _ [] yss = yss
        go qs (xs:xss) yss = go nextQs xss (ys:yss)
            where
                nextQs = L.eval xs qs <$> qdnfs
                ys = case machine of
                    MealyFSM -> L.eval xs qs <$> ydnfs
                    MooreFSM -> L.eval xs nextQs <$> ydnfs

evalStateDnfs :: [L.BoolTerm] -> [Enc]
evalStateDnfs qdnfs = eval qdnfs outputDnfs (replicate qwidth 0) checkInputs

printCheckResults outputs = do
    let matchStatus = if outputs == checkOutputs
        then "(matches expected)"
        else "(incorrect)"
    putStrLn $ "Inputs:  " ++ unwords ((show =<<) <$> checkInputs)
    putStrLn $ "Outputs: " ++ unwords ((show =<<) <$> outputs) ++ " " ++ matchStatus

# D flip-flop

In [ ]:
transitionsWithQ = zip (encodeBinaryAll qrange) <$> transitionsEncoded

dTransitions = (dTransition <$>) <$> transitionsWithQ
    where dTransition (_prevState, nextState) = nextState
dTransitionsWithXQ = zipWithXQ dTransitions

putStrLn $ tabulateSignals dTransitions [makeFooter 'D' qwidth]

qdnfrange = [0..qwidth - 1]
dDnfs = dnf dTransitionsWithXQ <$> qdnfrange
forM_ (zip [0..] dDnfs) $ \(d, qdnf) -> putStrLn $ renderDnf "D" d qdnf

dStateDnfs = dDnfs -- see dTransition
printCheckResults $ evalStateDnfs dStateDnfs

# T flip-flop

In [ ]:
tTransitions = (tTransition <$>) <$> transitionsWithQ
    where
        tTransition (prevState, nextState) = tBit <$> zip prevState nextState
        -- xor
        tBit (0, b) = b
        tBit (1, 0) = 1
        tBit (1, 1) = 0
tTransitionsWithXQ = zipWithXQ tTransitions

putStrLn $ tabulateSignals tTransitions [makeFooter 'T' qwidth]

tDnfs = dnf tTransitionsWithXQ <$> qdnfrange
forM_ (zip [0..] tDnfs) $ \(t, qdnf) -> putStrLn $ renderDnf "T" t qdnf

tStateDnfs = makeQdnf <$> zip [1..] tDnfs
    where
        makeQdnf (qi, tdnf) = xorTerm (L.Q qi) tdnf
        xorTerm a b = L.Or [L.And [L.Not a, b], L.And [a, L.Not b]]
printCheckResults $ evalStateDnfs tStateDnfs

# RS flip-flop

In [ ]:
import Data.List (intercalate)

tabulatePairSignals (avar, bvar) ss = tabulateSignals' (rows ++ footer)
    where
        rows = (intercalate "/" . (showPair <$>) <$>) <$> ss
            where
                showPair (a, b) = showBit a ++ showBit b
                showBit (-1) = "-"
                showBit n = show n
        footer = [replicate (length qrange) column]
            where
                column = intercalate "/ " $ signal <$> [1..qwidth]
                signal i = ['$', avar, '_'] ++ show i ++ [bvar, '_'] ++ show i ++ ['$']

In [ ]:
rsTransitions = (rsTransition <$>) <$> transitionsWithQ
    where
        rsTransition (prevState, nextState) = rsPair <$> zip prevState nextState
        rsPair (0, 0) = (-1, 0)
        rsPair (0, 1) = (0, 1)
        rsPair (1, 0) = (1, 0)
        rsPair (1, 1) = (0, -1)

putStrLn $ tabulatePairSignals ('R', 'S') rsTransitions

rDnfs = dnf (zipWithXQ rTransitions) <$> qdnfrange
    where rTransitions = ((fst <$>) <$>) <$> rsTransitions
sDnfs = dnf (zipWithXQ sTransitions) <$> qdnfrange
    where sTransitions = ((snd <$>) <$>) <$> rsTransitions
    
forM_ (zip3 [0..] rDnfs sDnfs) $ \(i, rdnf, sdnf) -> do
    putStrLn $ renderDnf "R" i rdnf
    putStrLn $ renderDnf "S" i sdnf
    
rsStateDnfs = makeQdnf <$> zip3 [1..] rDnfs sDnfs
    where
        makeQdnf (qi, rdnf, sdnf) = L.And [ L.Not rdnf
                                          , L.Or [ L.Q qi
                                                 , L.And [L.Not (L.Q qi), sdnf]]]

printCheckResults $ evalStateDnfs rsStateDnfs

# JK flip-flop

In [ ]:
jkTransitions = (jkTransition <$>) <$> transitionsWithQ
    where
        jkTransition (prevState, nextState) = jkPair <$> zip prevState nextState
        jkPair (0, 0) = (0, -1)
        jkPair (0, 1) = (1, -1)
        jkPair (1, 0) = (-1, 1)
        jkPair (1, 1) = (-1, 0)

putStrLn $ tabulatePairSignals ('J', 'K') jkTransitions

jDnfs = dnf (zipWithXQ jTransitions) <$> qdnfrange
    where jTransitions = ((fst <$>) <$>) <$> jkTransitions
kDnfs = dnf (zipWithXQ kTransitions) <$> qdnfrange
    where kTransitions = ((snd <$>) <$>) <$> jkTransitions
    
forM_ (zip3 [0..] jDnfs kDnfs) $ \(i, jdnf, kdnf) -> do
    putStrLn $ renderDnf "J" i jdnf
    putStrLn $ renderDnf "K" i kdnf

jkStateDnfs = makeQdnf <$> zip3 [1..] jDnfs kDnfs
    where
        makeQdnf (qi, jdnf, kdnf) = L.Or [notk, k]
            where
                notk = L.And [ L.Not kdnf
                             , L.Or [L.And [L.Not jdnf, L.Q qi], jdnf ]]
                k = L.And [kdnf, jdnf, L.Not (L.Q qi)]

printCheckResults $ evalStateDnfs jkStateDnfs